In [1]:
# boilerplate sanity check
import os,warnings,cpuinfo,multiprocessing,psutil,sys
warnings.simplefilter(action='ignore',category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
try:cpu=cpuinfo.get_cpu_info();print('CPU says hello!\n-name: {}\n-architecture: {}\n-bits: {}\n-frequency: {}\n-threads: {}\n-l3_cache: {}\n-l2_cache: {}\n-l1_cache: {}'.format(cpu['brand'],cpu['arch'],cpu['bits'],cpu['hz_actual'],multiprocessing.cpu_count(),cpu['l3_cache_size'],cpu['l2_cache_size'],cpu['l1_data_cache_size']))
except:print('Failed to get all CPU info!')
try:print('RAM says hello!\n-virtual: {} GB\n-swap: {} GB'.format(round(psutil.virtual_memory().total/1024**3,2),round(psutil.swap_memory().total/1024**3,2)))
except:print('Failed to get all RAM info!')
print('Python says hello!\n-python: {}'.format(sys.version.split(' ')[0]))
try:
    import tensorflow as tf;import keras;from tensorflow.python.client import device_lib;from tensorflow.keras import mixed_precision;from tensorflow.python.framework.ops import disable_eager_execution;gpus=[];
#     tf.config.set_visible_devices([], 'GPU')
    print('-tensorflow: {}'.format(tf.version.VERSION))
    print('-keras: {}'.format(keras.__version__))
    for device in device_lib.list_local_devices():
        try:
            if device.device_type=='GPU':
                name=device.name
                try:
                    splitted=device.physical_device_desc.split(',')
                    for part in splitted:
                        if'name:'in part or'Name:'in part:name=part.split(':')[1].strip()
                except:pass
                gpus.append('\n-name: {}\n-vram: {} GB'.format(name,round(device.memory_limit/1000**3,2)))
        except:pass
    if len(gpus)>0:
        print('GPU{} hello!{}'.format('s saying'if len(gpus)>1 else' says',''.join(gpus)))
        try:
            smi=os.popen('nvidia-smi').read().split('\n')[2];d=smi.find('Driver Version')+16;c=smi.find('CUDA Version')+14
            print('Drivers saying hello!\n-nvidia: {}\n-cuda: {}'.format(smi[d:d+10].strip(),smi[c:c+5].strip()))
        except:pass
    else:print('GPU not found!')
except:print('GPU not found! Check tensorflow-gpu and cudatoolkit installations!')
# print('===============================')
# try:
#     disable_eager_execution()
#     print('Eager execution disabled!')
# except:
#     print('Failed to disable eager execution!')
# try:
#     mixed_precision.set_global_policy('mixed_float16')
#     print('Mixed precision enabled!')
# except:
#     print('Failed to enable mixed precision!')

CPU says hello!
-name: 12th Gen Intel(R) Core(TM) i5-12600KF
-architecture: X86_64
-bits: 64
-frequency: 3.6865 GHz
-threads: 16
-l3_cache: 16384 KB
-l2_cache: 64 MiB
-l1_cache: 512 KiB
RAM says hello!
-virtual: 102.17 GB
-swap: 256.0 GB
Python says hello!
-python: 3.8.10


2024-10-23 18:33:49.132545: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


-tensorflow: 2.10.0
-keras: 2.10.0
GPU says hello!
-name: Tesla P40
-vram: 23.21 GB
Drivers saying hello!
-nvidia: 560.35.03
-cuda: 12.6


In [2]:
import os, warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

import sys
sys.path.insert(1,'source')
from DataGenerator import DataGenerator
import numpy as np

In [3]:
datagen_props = {
    'path'          : 'source/data',     #path of the data
    'seed'          : 42,         #seed for the split
    'split'         : 0.8,        #train/all ratio
    'train'         : True,       #training/testing split
    'control'       : True,       #include control data points
    'huntington'    : False,      #include huntington data points
    'batch_size'    : 50000,      #batch size
    'spatial'       : False,      #keep spaital format of flatten voxels in the brain region
    'left'          : True,       #include left hemisphere data (if both false, concatenate the left and right hemisphere layers)
    'right'         : True,       #include right hemisphere data
    'threshold'     : 0.5,        #if float value provided, it thresholds the connectivty map
    'binarize'      : False,      #only works if threshold if greater or equal than half, and then it binarizes the connectivity map
    'not_connected' : False,      #only works if thresholded and not single, and then it appends an extra encoding for the 'not connected'
    'single'        : 0,          #if int index value is provided, it only returns a specified connectivity map
    'target'        : False,
    'roi'           : False,
    'brain'         : False,
    'features'      : [],         #used radiomics features (emptylist means all)
    'features_vox'  : [],         #used voxel based radiomics features (emptylist means all)
    'radiomics'     : ['b25'],    #used radiomics features bin settings
    'radiomics_vox' : ['k5_b25'], #used voxel based radiomics features kernel and bin settings
    'pre_load_all'  : True,
}

In [4]:
datagen_props1 = datagen_props.copy()
train = DataGenerator(**datagen_props1)
datagen_props2 = datagen_props.copy()
datagen_props2['train'] = False
test = DataGenerator(**datagen_props2)

In [11]:
model = Sequential(name='FFN')
model.add(Input(train.x_shape))
model.add(Dense(512,activation='silu'))
model.add(Dense(512,activation='silu'))
model.add(Dense(512,activation='silu'))
model.add(Dense(512,activation='silu'))
model.add(Dense(128,activation='silu'))
model.add(Dense(64,activation='silu'))
model.add(Dense(train.y_shape[0],activation='silu'))
#model.summary()

model.compile(loss='mean_absolute_error', optimizer='adam')
history = model.fit(
    train,
    steps_per_epoch=train.steps,
    validation_data=test,
    validation_steps=test.steps,
    epochs=20,
    verbose=1,
)

Epoch 1/20


 1/14 [=>............................] - ETA: 7s - loss: 0.0434

 2/14 [===>..........................] - ETA: 1s - loss: 0.1046

 3/14 [=====>........................] - ETA: 0s - loss: 0.1084

 4/14 [=======>......................] - ETA: 0s - loss: 0.0895

...

11/14 [======================>.......] - ETA: 0s - loss: 0.0523

12/14 [========================>.....] - ETA: 0s - loss: 0.0502

13/14 [==========================>...] - ETA: 0s - loss: 0.0497

14/14 [==============================] - ETA: 0s - loss: 0.0500

14/14 [==============================] - 1s 103ms/step - loss: 0.0500 - val_loss: 0.0548


In [10]:
np.count_nonzero(train.data[0][1]) / np.prod(train.data[0][1].shape)

0.04566